In [1]:
import cv2
import torch
from tracker import *
import numpy as np
count = set() # to store the ID of a person 
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)
cap=cv2.VideoCapture("C:/Users/ASUS/Downloads/project/cctv.mp4")

def print_mouse_position(event, x, y, flags, param):
    if event == cv2.EVENT_MOUSEMOVE :  
        position = [x, y]
        print(position)
        
cv2.namedWindow('FRAME')
#cv2.setMouseCallback('FRAME', position) # to identify area by mouse event

tracker = Tracker()
area = [(377,315),(429,373),(535,339),(500,296)]
while True:
    ret,frame=cap.read()
    frame=cv2.resize(frame, (1020, 500))
    cv2.polylines(frame,[np.array(area,np.int32)] ,True, (0, 255, 0), 2)
    results = model(frame)
    #frame = np.squeeze(results.render())
    points = [] # to store the coordinates of a person
    for index , row in results.pandas().xyxy[0].iterrows():
        x1, y1, x2, y2 = map(int, [row['xmin'], row['ymin'], row['xmax'], row['ymax']])
        n=(row['name'])
        if 'person' in n:
            points.append([x1, y1, x2, y2]) 
            #cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 255), 2)
            #cv2.putText(frame, str(n), (x1, y1), cv2.FONT_HERSHEY_PLAIN, 2, (255, 0, 0), 2)

    boxes_id = tracker.update(points) # make a unique ID for each person  
    #print(boxes_id)
    for box_id in boxes_id:
        x, y, w, h, idd = box_id
        cv2.rectangle(frame,(x, y), (w, h), (255, 0, 255), 2)
        cv2.putText(frame,str(idd),(x,y),cv2.FONT_HERSHEY_PLAIN,3,(255,0,0),2)
        if cv2.pointPolygonTest(np.array(area,np.int32), (w,h), False)>=0:
            count.add(idd)
    #print(c) 
    sum = len(count)
    cv2.putText(frame,'number of persons is ='+str(sum), (50,65), cv2.FONT_HERSHEY_PLAIN, 3, (0,0,0),4)
    
    cv2.imshow('FRAME',frame)
    if cv2.waitKey(15)&0xFF==27: # because frame rate for that video 15 frames/second
        break
cap.release()
cv2.destroyAllWindows()

Using cache found in C:\Users\ASUS/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2024-2-10 Python-3.11.5 torch-2.2.0+cu118 CUDA:0 (NVIDIA GeForce RTX 2060, 6144MiB)

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 
